In [23]:
import numpy as np
import random
import os
os.environ['ON_PY']="1"


In [24]:
from params import configs
from data_utils import SD2_instance_generator
num_jobs = 20
num_machines = 5
num_operations = 5

job_length, op_pt, op_per_mch = SD2_instance_generator(configs, num_jobs, num_machines, num_operations)
job_length



array([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5])

In [25]:

# 定义问题的输入
processing_times = np.array(op_pt)

# 初始化种群
def initialize_population(pop_size, num_jobs, num_machines):
    population = []
    for _ in range(pop_size):
        individual = []
        for job in range(num_jobs):
            for op in range(num_operations):
                machines = [m for m in range(num_machines) if processing_times[job * num_operations + op, m] > 0]
                machine_choice = random.choice(machines)
                individual.append((job, op, machine_choice))
        random.shuffle(individual)
        population.append(individual)
    return population

# 计算适应度
def calculate_fitness(individual):
    machine_time = np.zeros(num_machines)
    job_time = np.zeros(num_jobs)
    
    for job, op, machine in individual:
        start_time = max(machine_time[machine], job_time[job])
        processing_time = processing_times[job * num_operations + op, machine]
        machine_time[machine] = start_time + processing_time
        job_time[job] = start_time + processing_time
    
    return max(machine_time)

# 选择
def selection(population, fitness):
    sorted_population = [x for _, x in sorted(zip(fitness, population))]
    return sorted_population[:len(population)//2]

# 交叉
def crossover(parent1, parent2):
    point = random.randint(1, len(parent1) - 2)
    child1 = parent1[:point] + parent2[point:]
    child2 = parent2[:point] + parent1[point:]
    return child1, child2

# 变异
def mutate(individual, mutation_rate):
    if random.random() < mutation_rate:
        i, j = random.sample(range(len(individual)), 2)
        individual[i], individual[j] = individual[j], individual[i]

# 遗传算法
# 遗传算法
def genetic_algorithm(pop_size, mutation_rate, generations):
    population = initialize_population(pop_size, num_jobs, num_machines)
    best_fitness_over_time = []
    
    for _ in range(generations):
        fitness = [calculate_fitness(ind) for ind in population]
        best_fitness_over_time.append(min(fitness))
        population = selection(population, fitness)
        
        next_generation = []
        while len(next_generation) < pop_size:
            parent1, parent2 = random.sample(population, 2)
            child1, child2 = crossover(parent1, parent2)
            mutate(child1, mutation_rate)
            mutate(child2, mutation_rate)
            next_generation.extend([child1, child2])
        
        population = next_generation
    
    fitness = [calculate_fitness(ind) for ind in population]
    best_individual = population[np.argmin(fitness)]
    best_fitness_over_time.append(min(fitness))
    
    return best_individual, min(fitness), best_fitness_over_time


Time:3.277974843978882
Best solution: [(0, 2, 2), (6, 4, 0), (0, 2, 2), (0, 2, 2), (1, 2, 0), (1, 0, 2), (4, 2, 0), (7, 4, 4), (0, 4, 0), (1, 2, 0), (1, 2, 0), (2, 1, 3), (8, 4, 2), (5, 1, 2), (2, 4, 3), (2, 1, 3), (6, 4, 4), (2, 1, 3), (6, 3, 2), (2, 4, 3), (2, 1, 3), (5, 3, 4), (5, 2, 3), (3, 0, 1), (2, 1, 3), (0, 2, 2), (0, 2, 2), (3, 0, 1), (5, 2, 3), (7, 4, 4), (1, 3, 1), (3, 0, 1), (5, 1, 2), (11, 0, 0), (6, 4, 0), (1, 2, 0), (2, 3, 1), (12, 1, 4), (1, 1, 0), (6, 4, 4), (11, 2, 3), (13, 3, 1), (8, 4, 2), (5, 3, 4), (1, 2, 0), (4, 1, 1), (5, 3, 4), (19, 2, 0), (8, 4, 2), (3, 1, 3), (1, 0, 2), (1, 2, 0), (0, 2, 2), (6, 2, 2), (0, 4, 0), (1, 3, 1), (5, 3, 4), (1, 3, 1), (6, 4, 0), (1, 2, 0), (7, 4, 4), (2, 3, 1), (12, 1, 3), (10, 1, 2), (10, 3, 3), (4, 1, 1), (4, 1, 3), (14, 2, 0), (1, 2, 0), (6, 3, 2), (2, 1, 3), (1, 0, 2), (8, 4, 2), (12, 4, 0), (7, 4, 4), (4, 1, 3), (1, 3, 1), (5, 2, 3), (5, 2, 3), (1, 3, 1), (1, 0, 2), (7, 4, 4), (6, 4, 0), (6, 0, 0), (6, 4, 0), (12, 1, 4), (2, 

In [26]:

# 设置参数并运行遗传算法
population_size = 20
mutation_rate = 0.1
generations = 2500

import time
import matplotlib.pyplot as plt

t1 = time.time()
best_solution, best_fitness, fitness_over_time = genetic_algorithm(population_size, mutation_rate, generations)
t2 = time.time()
print(f"Time:{t2 - t1}")
print(f"Best solution: {best_solution}")
print(f"Best fitness (makespan): {best_fitness}")


# 可视化迭代过程中的makespan
plt.plot(fitness_over_time)
plt.title('Genetic Algorithm: Makespan over Generations')
plt.xlabel('Generation')
plt.ylabel('Best Makespan')
plt.grid(True)
plt.show()


